In [ ]:
from __future__ import division
from ADMCode import visualize as vis
from ADMCode import ddm, sdt

import numpy as np
import pandas as pd

from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.simplefilter('ignore', np.RankWarning)
warnings.filterwarnings("ignore", module="matplotlib")
warnings.filterwarnings("ignore")
sns.set(style='white', font_scale=1.3)

%matplotlib inline

# Signal Detection Theory

In [ ]:
interactive_plot = interactive(vis.sdt_interact, pH=(0.,1.,.1), pFA=(0.,1.,.1))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

#  Questions on SDT: 
#### (double click on the cells below to edit)

#### **Question 1:**  

**1a)** Describe the relationship between number of **Misses** and the criterion parameter ($c$) in SDT (use the interactive visualization at the top to help get some intuition). 

* **Answer 1a:** 



**1b)** Describe the relationship between number of **Misses** and $d'$. 

* **Answer 1b:** 



**1c)** Describe in plain words why $c=\frac{1}{2}d'$ when the **Hit** and **Miss** counts are equal. 

* **Answer 1c:** 



#  Questions on DDM: 
#### (see code below)
####  (double click on the cells below to edit)

#### **Question 2** 
---

One way to model **signal** and **noise** trials is in a "yes"/"no" experiment in which true "signals" cause evidence to accumulate with positive drift ($v_s>0$) and noise cause evidence to accumulate with negative drift ($v_n<0$). In that experiment, the "subject" is required to provide a response on every trial - "yes" if they detected a signal and "no" if they only detected noise. 

There are many real world examples of a similar, but slightly different behavioral task - in which an action (i.e., reporting "yes") is ONLY required if a signal is detected and to simply withhold responding otherwise. 

Think about the radar example: a person monitoring a radar screen for enemy ships has the single goal to launch a missile if and only if they detect an enemy ship ("signal"). and to withhold their response otherwise.  


---
**2a** Describe how you would modify the parameters of the DDM to formally model this simple detection experiment. You can remove or add any parameters but must provide a reasonable justification for your proposed modifications. Remember, adding too many parameters will make your model intractable and you should try to seek an elegant solution (e.g., avoid unnecessary complexity)

* **Answer 2a:** 

**2b)** Implement the model you proposed in your answer to **Question 2a**. You can choose to create a new function called 'sim_mod_ddm()' or to edit the existing function to allow for the go/no-go experiment. 


#### **Question 3** 
---
Try to understand the different parameters of the DDM better. Run some simulations in which you modify drift rate $v$, boundary height $a$, noise $s_i$ and starting point $z$. How do these parameters affect the resulting RT and choice distributions? 

# Drift-Diffusion Model

In [ ]:
from numpy.random import random_sample

def sim_ddm(parameters, deadline=2.0):
    """
    ::Arguments::
        parameters: 1d array (Nparams) of DDM parameters
            a: boundary height
            tr: non-decision time
            v: drift-rate
            z: starting point (frac of a; 0.0 < z < 1.0)
            si: diffusion constant (sigma param in DDM equation from lecture)
            dx: step-size of evidence
            dt: time step
        rProb: 1d array (Ntimesteps) of random floats between 0 and 1
        trace: 1d array (Ntimesteps) for storing the evidence trace
    
    ::Returns::
        RT (float): the time that evidence crossed one of the boundaries
        choice: 1 if evidence terminated at upper bound, 0 if lower bound
    """
    
    # extract parameters
    a, tr, v, z, si, dx, dt = parameters
    
    # convert drift-rate into a probability, 
    # & scale by sigma (si) and timestep (dt)
    # if v > 0, then 0.5 < vProb < 1.0
    # if v < 0, then 0.0 < vProb < 0.5
    vProb = .5 * (1 + (v * np.sqrt(dt))/si)
    # define starting point with respect to boundary height
    zStart = z * a
    #initialize evidence variable at zStart
    evidence = zStart
    max_steps = int(deadline / dt)
    trace = np.zeros(max_steps)
    trace[0] = evidence
    
    # define deadline (max time allowed for accumulation)
    #deadline = trace.size

    for nsteps in range(1, max_steps):
        # sample a random probability (r) and compare w/ vProb
        if random_sample() < vProb:
            # if r < vProb, step evidence up (towards a)
            evidence += dx
        else:
            # if r > vProb, step evidence down (towards 0)
            evidence -= dx
        # store new value of evidence at current timestep
        trace[nsteps] = evidence

        # check if new value of evidence crossed bound
        if evidence >= a:
            # calculate RT (in milliseconds)
            rt = tr + (nsteps * dt)
            # set choice to 1.0 (upper bound)
            choice = 1.0
            # terminate simulation
            break

        elif evidence <= 0:
            # calculate RT (in milliseconds)
            rt = tr + (nsteps * dt)
            # set choice to 0.0 (lower bound)
            choice = 0.0
            # terminate simulation
            break
            
    plt.plot(trace[:nsteps+1])
    return rt, choice

## Build your modified DDM

In [ ]:
def sim_mod_ddm(parameters, deadline=2.0):
    """
    modified function to run for a go/no-go experiment
    you can start by copying and pasting the code for sim_ddm
    """
    raise NotImplementedError() # get rid of this after completing function
    return rt, choice

## Define Parameters

In [ ]:
a = .15 # boundary height
v = .24 # strong drift-rate
tr = .25 # nondecision time (in seconds)
z = .5 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .1 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 1.75 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 1000 # number of trials to simulate

parameters = np.array([a, tr, v, z, si, dx, dt])

## Simulate standard DDM

In [ ]:
rts, choices = [], []
for trial in range(ntrials):
    rt, choice = sim_ddm(parameters, deadline)
    rts.append(rt)
    choices.append(choice)
rts = np.array(rts)
print(f'mean RT for "yes": {np.mean(rts[np.equal(choices, 1)])}')
print(f'mean RT for "no": {np.mean(rts[np.equal(choices,0)])}')
print(f'fraction of "go" responses: {np.mean(choices)}')

## Simulate modified DDM

In [ ]:
a = .15 # boundary height
v = .24 # strong drift-rate
tr = .25 # nondecision time (in seconds)
z = 0 # starting point ([0,1], fraction of a)

dt = .001 # time step
si = .1 # sigma (noise scalar)
dx = si * np.sqrt(dt) # evidence step
deadline = 1 # max decision time
ntime = np.int(np.floor(deadline / dt)) # time limit for decision
ntrials = 1000 # number of trials to simulate

parameters = np.array([a, tr, v, z, si, dx, dt])

rts, choices = [], []
for trial in range(ntrials):
    rt, choice = sim_mod_ddm(parameters, deadline)
    rts.append(rt)
    choices.append(choice)
rts = np.array(rts)
print(f'mean RT for "go": {np.mean(rts[np.logical_not(np.isnan(rts))])}')
print(f'fraction of "go" responses: {np.mean(choices)}')

## Do some simulations here (for question 3)